<a href="https://colab.research.google.com/github/Jegovila/cursoVR/blob/main/Practica2%3A%20Histogramas/python/Pr%C3%A1ctica2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica2%3A%20Histogramas/data/im1.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica2%3A%20Histogramas/data/im2.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica1%3A%20Fundamentos%20del%20PDI/data/lena.bmp"

In [2]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

# Imagen inversa

In [ ]:
src = cv2.imread("lena.bmp")
src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)

negativo = 255 - src

fig, ax = plt.subplots(1, 2, figsize=(20,14))
fig.tight_layout()

plt.subplot(1,2,1)
plt.imshow(src)
plt.subplot(1,2,2)
plt.imshow(negativo)

# Desplegar histogramas

In [ ]:
src = cv2.imread("lena.bmp")
cv2_imshow(src)
color = ('b', 'g', 'r')
for i, col in enumerate(color):
    histr = cv2.calcHist([src], [i], None, [256], [0, 256])
    plt.plot(histr, color=col)
    plt.xlim([0, 256])
plt.draw()
plt.pause(0.01)

# Modificar contraste y brillo

In [7]:
src = cv2.imread("lena.bmp")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
graymod = gray * 0.3 + 80
graymod = graymod.astype('uint8')

Desplegar resultados

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20,14))
fig.tight_layout()

plt.subplot(2,2,1)
plt.imshow(gray, cmap='gray')
plt.title('Original')
plt.subplot(2,2,2)
plt.plot(cv2.calcHist([gray], [0], None, [256], [0, 256]))
plt.title('histograma original')
plt.subplot(2,2,3)
# aqui establecemos los valores min y max como 0 y 255
# de lo contrario, plt.imshow mapeará el minimo a 0 y el
# máximo a 255 y no veríamos la diferencia en el contraste
plt.imshow(graymod, cmap="gray", vmin=0, vmax=255)
plt.title('Modificada')
plt.subplot(2,2,4)
plt.plot(cv2.calcHist([graymod], [0], None, [256], [0, 256]))
plt.title('histograma modificado')

# Adaptación automática del contraste

In [9]:
src = cv2.imread("lena.bmp")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
graymod = gray * 0.3 + 80
graymod = graymod.astype('uint8')

bajo = graymod.min()
alto = graymod.max()
grayad = np.zeros((gray.shape[0], gray.shape[1], 1), np.uint8)
for i in range(gray.shape[0]):
    for j in range(gray.shape[1]):
        grayad[i][j] = ((graymod[i][j] - bajo) / (alto - bajo) ) * 255
grayad = grayad.astype('uint8')

Desplegar resultados

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20,14))
fig.tight_layout()

plt.subplot(2,2,1)
plt.imshow(graymod, cmap="gray", vmin=0, vmax=255)
plt.title('Original')
plt.subplot(2,2,2)
plt.plot(cv2.calcHist([graymod], [0], None, [256], [0, 256]))
plt.title('histograma original')
plt.subplot(2,2,3)
plt.imshow(grayad, cmap="gray", vmin=0, vmax=255)
plt.title('Modificada')
plt.subplot(2,2,4)
plt.plot(cv2.calcHist([grayad], [0], None, [256], [0, 256]))
plt.title('histograma modificado')

In [ ]:
print("Imagen original ->  vmin: ", np.min(graymod), " vmax: ", np.max(graymod))
print("Imagen adaptada ->  vmin: ", np.min(grayad), " vmax: ", np.max(grayad))

# Ecualización lineal

## Función para generar histograma acumulado

In [12]:
def acumulado(img):
    gray_hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    # Acumulado:
    acc = 0
    H = np.zeros(256)
    for i in range(256):
        H[i] = gray_hist[i] + acc
        acc = H[i]
    return H

## Ecualización lineal

In [13]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
H = acumulado(gray)

# Ecualización
Ieq = np.zeros((gray.shape[0], gray.shape[1], 1), np.uint8)
for i in range(gray.shape[0]):
    for j in range(gray.shape[1]):
        v = gray[i][j]
        Ieq[i][j] = H[gray[i][j]] * (255/(gray.shape[0]*gray.shape[1]))

# Acumulado eq:
Heq = acumulado(Ieq)

Desplegar resultados

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20,14))
fig.tight_layout()

plt.subplot(2,2,1)
plt.imshow(gray, cmap="gray", vmin=0, vmax=255)
plt.title('Original')
plt.subplot(2,2,2)
plt.plot(H)
plt.title('histograma original')
plt.subplot(2,2,3)
plt.imshow(Ieq, cmap="gray", vmin=0, vmax=255)
plt.title('Modificada')
plt.subplot(2,2,4)
plt.plot(Heq)
plt.title('histograma modificado')

# Ecualización por especificación

## Histograma acumulado

In [18]:
def acumulado(img):
    H = np.zeros([256, 3])
    for i, col in enumerate(color):
        hist = cv2.calcHist([img], [i], None, [256], [0, 256])
        # Acumulado:
        acc = 0
        for j in range(256):
            H[j][i] = hist[j] + acc
            acc = H[j][i]
        #Normalizado
        H[:,i] = H[:,i] / (img.shape[0] * img.shape[1])
        # plt.figure(figure)
        # plt.plot(H[:,i],color=col)
        # plt.draw()
        # plt.pause(.001)
    return H

## Ecualización por especificación

In [27]:
I1 = cv2.imread('im1.jpg')
I2 = cv2.imread('im2.jpg')

I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2RGB)
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2RGB)

H1 = acumulado(I1)
H2 = acumulado(I2)

[m, n, p] = I1.shape
I3 = np.zeros([m, n, p])

#--------Ecualización por especificación----------
for k in range(p):
    for i in range(m):
        for j in range(n):
            z = 0
            Sk = H1[I1[i, j, k], k]
            while H2[z, k] - Sk < 0:
                z += 1
                if z == 256:
                    H2[z, k] = Sk
            I3[i, j, k] = z

I3 = I3.astype('uint8')
H3 = acumulado(I3)

Desplegar resultados

In [ ]:
color = ('r', 'g', 'b')
fig, ax = plt.subplots(2, 3, figsize=(20,14))
fig.tight_layout()

plt.subplot(2,3,1)
plt.imshow(I1)
plt.title('src')
plt.subplot(2,3,2)
plt.imshow(I2)
plt.title('dst')
plt.subplot(2,3,3)
plt.imshow(I3)
plt.title('out')
plt.subplot(2,3,4)
for i, col in enumerate(color):
  plt.plot(H1[:,i],color=col)
plt.subplot(2,3,5)
for i, col in enumerate(color):
  plt.plot(H2[:,i],color=col)
plt.subplot(2,3,6)
for i, col in enumerate(color):
  plt.plot(H3[:,i],color=col)

# Alpha Blending

In [37]:
I1 = cv2.imread('im1.jpg')
I2 = cv2.imread('im2.jpg')
I2 = cv2.resize(I2, (I1.shape[1], I1.shape[0]))

I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2GRAY)
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)

alpha_value = 0.5

I3 = cv2.addWeighted(I1, alpha_value, I2, 1-alpha_value, 0)

Desplegar resultados

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,14))
fig.tight_layout()

plt.subplot(1,3,1)
plt.imshow(I1, cmap="gray")
plt.subplot(1,3,2)
plt.imshow(I2, cmap="gray")
plt.subplot(1,3,3)
plt.imshow(I3, cmap="gray")